## Walt Disney World Resort Areas Part 3

### Segmenting and Clustering WDW Resort areas

Import necessary libraries.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Load and explore the data

In [2]:
resorts=pd.read_csv('wdw_resorts_geo.csv')

In [3]:
resorts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 7 columns):
Resort          31 non-null object
Opening Date    31 non-null object
Theme           31 non-null object
Rooms           31 non-null object
Resort Area     31 non-null object
Latitude        31 non-null float64
Longitude       31 non-null float64
dtypes: float64(2), object(5)
memory usage: 1.8+ KB


In [4]:
resorts

,Resort,Opening Date,Theme,Rooms,Resort Area,Latitude,Longitude
0,Disney's Animal Kingdom Lodge,"April 16, 2001",African Wildlife preserve,"1,307",Animal Kingdom,28.352550,-81.603430
1,Disney's Beach Club Resort,"November 19, 1990",Newport Beach cottage,576,Epcot,28.373820,-81.557310
2,Disney's BoardWalk Inn,"July 1, 1996",Early-20th-century Atlantic and Ocean City,378,Epcot,28.373820,-81.557310
3,Disney's Contemporary Resort,"October 1, 1971",Modern,655,Magic Kingdom,28.416410,-81.573630
4,Disney's Grand Floridian Resort & Spa,"July 1, 1988",Early-20th-century Florida,867,Magic Kingdom,28.410810,-81.588140
5,Disney's Polynesian Village Resort,"October 1, 1971",South Seas,492,Magic Kingdom,28.398240,-81.582530
6,Disney's Wilderness Lodge,"May 28, 1994","Pacific Northwest, National Park Service rustic",729,Magic Kingdom,28.407089,-81.573532
7,Disney's Yacht Club Resort,"November 5, 1990",Martha's Vineyard Resort,621,Epcot,28.373820,-81.557310
8,Disney's Caribbean Beach Resort,"October 1, 1988",Caribbean Islands,"1,536",Epcot,28.362020,-81.542640
9,Disney's Coronado Springs Resort,"August 1, 1997","Mexico, American Southwest","1,915",Animal Kingdom,28.357890,-81.572490


#### Use geopy library to get the latitude and longitude values of Lake Buena Vista, Florida USA.

In order to define an instance of the geocoder, we need to define a user_agent. Agent is named <em>wdw_explorer</em>, as shown below.

In [5]:
address = 'Lake Buena Vista, Florida'

geolocator = Nominatim(user_agent="wdw_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lake Buena Vista, Florida are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Lake Buena Vista, Florida are 28.38034955, -81.5297578369267.


#### Create a map of Lake Buena Vista, FL USA with resorts superimposed on top.

In [6]:
# create map of Lake Buena Vista, FL using latitude and longitude values
map_wdw = folium.Map(location=[latitude, longitude], zoom_start=12.5)

# add markers to map
for lat, lng, resort_area, resort in zip(resorts['Latitude'], resorts['Longitude'], 
                                         resorts['Resort Area'], resorts['Resort']):
    label = '{}, {}'.format(resort, resort_area)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_wdw)  
    
map_wdw

Segment and cluster only the resorts in the Epcot resort area. 

In [7]:
epcot_resorts=resorts[resorts['Resort Area'].str.contains('Epcot')].reset_index(drop=True)
epcot_resorts

,Resort,Opening Date,Theme,Rooms,Resort Area,Latitude,Longitude
0,Disney's Beach Club Resort,"November 19, 1990",Newport Beach cottage,576,Epcot,28.373820,-81.557310
1,Disney's BoardWalk Inn,"July 1, 1996",Early-20th-century Atlantic and Ocean City,378,Epcot,28.373820,-81.557310
2,Disney's Yacht Club Resort,"November 5, 1990",Martha's Vineyard Resort,621,Epcot,28.373820,-81.557310
3,Disney's Caribbean Beach Resort,"October 1, 1988",Caribbean Islands,"1,536",Epcot,28.362020,-81.542640
4,Disney's Beach Club Villas,"July 1, 2002",Newport resort,282,Epcot,28.373820,-81.557310
5,Disney's BoardWalk Villas,"July 1, 1996",Early-20th-century Atlantic City,530,Epcot,28.373819,-81.557312
6,Disney's Riviera Resort,"December 16, 2019",European Riviera,300,Epcot,28.377970,-81.519040


In [8]:
epcot_resorts.shape

(7, 7)

#### Visualize resorts in the Epcot area.

First, get latitude and longitude for Epcot.

In [13]:
from geopy.geocoders import TomTom
geolocator = TomTom('gyZ64VaXG4MsLuF9qOtWlOvPBGzGAcQO')
epcot_address='200 Epcot Center Dr, Orlando, FL 32821'
address, (latitude, longitude) = geolocator.geocode(epcot_address)
print('The Epcot address is ', epcot_address, ', and the coordinates are ', latitude, longitude)

The Epcot address is  200 Epcot Center Dr, Orlando, FL 32821 , and the coordinates are  28.35934 -81.5189


#### Create map of Epcot resort area with markers for Epcot resorts.

In [14]:
# create map of the Epcot area using latitude and longitude values
map_epcot_resorts = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, label in zip(epcot_resorts['Latitude'], epcot_resorts['Longitude'], epcot_resorts['Resort']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_epcot_resorts)  
    
map_epcot_resorts

### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [15]:
CLIENT_ID = 'U1FUA0IEYYHX3BE3CEQHSDUIIC2RQWGEAPIXSEDMR55TZSIL' # your Foursquare ID
CLIENT_SECRET = 'SHI5WKHCX1X0FE1AYUDQREAWEERWYXJSO5KUZ2JNMHAGKCD3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: U1FUA0IEYYHX3BE3CEQHSDUIIC2RQWGEAPIXSEDMR55TZSIL
CLIENT_SECRET:SHI5WKHCX1X0FE1AYUDQREAWEERWYXJSO5KUZ2JNMHAGKCD3


#### Explore the first resort in our dataframe.

Get the first resort.

In [16]:
epcot_resorts.loc[0, 'Resort']

"Disney's Beach Club Resort"

Get the latitude and longitude values for the first resort.

In [17]:
resort_latitude = epcot_resorts.loc[0, 'Latitude'] # neighborhood latitude value
resort_longitude = epcot_resorts.loc[0, 'Longitude'] # neighborhood longitude value

resort_name = epcot_resorts.loc[0, 'Resort'] # borough name

print('Latitude and longitude values of {} are {}, {}.'.format(resort_name,
                                                               resort_latitude,
                                                               resort_longitude))

Latitude and longitude values of Disney's Beach Club Resort are 28.37382, -81.55731.


#### Now, let's get the top 100 venues that are within a radius of 500 meters of the first resort.

First, let's create the GET request URL. Name your URL **url**.

#### Increased radius from 500 to 1000 to try to get more diverse venue results for the different resorts.

In [18]:
LIMIT=100
radius=1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    resort_latitude, 
    resort_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=U1FUA0IEYYHX3BE3CEQHSDUIIC2RQWGEAPIXSEDMR55TZSIL&client_secret=SHI5WKHCX1X0FE1AYUDQREAWEERWYXJSO5KUZ2JNMHAGKCD3&v=20180605&ll=28.37382,-81.55731&radius=1000&limit=100'

Send the GET request and examine the resutls

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e1263ae0be7b4001b4f7812'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Walt Disney World Resort',
  'headerFullLocation': 'Walt Disney World Resort, Bay Lake',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 158,
  'suggestedBounds': {'ne': {'lat': 28.382820009000007,
    'lng': -81.54710025564435},
   'sw': {'lat': 28.36481999099999, 'lng': -81.56751974435565}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b119bddf964a5207a8023e3',
       'name': "Disney's Beach Club Resort",
       'location': {'address': '1800 Epcot Resorts Blvd',
        'crossStreet': 'Walt Disney World',
   

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Disney's Beach Club Resort,Resort,28.371218,-81.556503
1,Cape May Cafe,Buffet,28.371057,-81.556779
2,Disney's Yacht Club Resort,Resort,28.370957,-81.558713
3,Yachtsman Steakhouse,Steakhouse,28.371014,-81.558146
4,Martha's Vineyard Lounge,Hotel Bar,28.371141,-81.557083


And how many venues were returned by Foursquare?

In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## 2. Explore resorts in the Epcot area

#### Let's create a function to repeat the same process to all the resorts in the Epcot area.

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Resort', 
                  'Resort Latitude', 
                  'Resort Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called 'toronto_venues*.

In [24]:
epcot_resort_venues = getNearbyVenues(names=epcot_resorts['Resort'],
                                   latitudes=epcot_resorts['Latitude'],
                                   longitudes=epcot_resorts['Longitude'])

Disney's Beach Club Resort
Disney's BoardWalk Inn
Disney's Yacht Club Resort
Disney's Caribbean Beach Resort
Disney's Beach Club Villas
Disney's BoardWalk Villas
Disney's Riviera Resort


#### Let's check the size of the resulting dataframe

In [25]:
print(epcot_resort_venues.shape)
epcot_resort_venues.head()

(156, 7)


,Resort,Resort Latitude,Resort Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Disney's Beach Club Resort,28.37382,-81.55731,Disney's Beach Club Resort,28.371218,-81.556503,Resort
1,Disney's Beach Club Resort,28.37382,-81.55731,Cape May Cafe,28.371057,-81.556779,Buffet
2,Disney's Beach Club Resort,28.37382,-81.55731,Disney's Yacht Club Resort,28.370957,-81.558713,Resort
3,Disney's Beach Club Resort,28.37382,-81.55731,Martha's Vineyard Lounge,28.371141,-81.557083,Hotel Bar
4,Disney's Beach Club Resort,28.37382,-81.55731,Yachtsman Steakhouse,28.371014,-81.558146,Steakhouse


In [26]:
epcot_resort_venues.columns

Index(['Resort', 'Resort Latitude', 'Resort Longitude', 'Venue',
       'Venue Latitude', 'Venue Longitude', 'Venue Category'],
      dtype='object')

Let's check how many venues were returned for each borough.

In [27]:
epcot_resort_venues.groupby('Resort').count()

,Resort Latitude,Resort Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Resort,,,,,,
Disney's Beach Club Resort,18,18,18,18,18,18
Disney's Beach Club Villas,18,18,18,18,18,18
Disney's BoardWalk Inn,18,18,18,18,18,18
Disney's BoardWalk Villas,18,18,18,18,18,18
Disney's Caribbean Beach Resort,34,34,34,34,34,34
Disney's Riviera Resort,32,32,32,32,32,32
Disney's Yacht Club Resort,18,18,18,18,18,18


#### Let's find out how many unique categories can be curated from all the returned venues

In [28]:
print('There are {} uniques categories.'.format(len(epcot_resort_venues['Venue Category'].unique())))

There are 37 uniques categories.


<a id='item3'></a>

## 3. Analyze Epcot resorts

In [29]:
# one hot encoding
epcot_resort_onehot = pd.get_dummies(epcot_resort_venues[['Venue Category']], prefix="", prefix_sep="")

# add resort column back to dataframe
epcot_resort_onehot['Resort'] = epcot_resort_venues['Resort'] 

# move resort column to the first column
col = epcot_resort_onehot['Resort']
epcot_resort_onehot.drop(labels=['Resort'], axis=1,inplace = True)
epcot_resort_onehot.insert(0, 'Resort', col)

epcot_resort_onehot.head()

,Resort,American Restaurant,Arcade,Beach,Bistro,Boat or Ferry,Boutique,Buffet,Bus Station,Bus Stop,Café,Coffee Shop,Convenience Store,Fast Food Restaurant,Food,General Entertainment,Golf Course,Gym,Gym / Fitness Center,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Laundromat,Lighthouse,Lounge,Playground,Pool Hall,Rental Car Location,Restaurant,Spa,Steakhouse,Theater,Theme Park Ride / Attraction,Theme Restaurant,Tourist Information Center,Tram Station
0,Disney's Beach Club Resort,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Disney's Beach Club Resort,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Disney's Beach Club Resort,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Disney's Beach Club Resort,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Disney's Beach Club Resort,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


And let's examine the new dataframe size.

In [30]:
epcot_resort_onehot.shape

(156, 37)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [31]:
epcot_resort_grouped = epcot_resort_onehot.groupby('Resort').mean().reset_index()
epcot_resort_grouped

,Resort,American Restaurant,Arcade,Beach,Bistro,Boat or Ferry,Boutique,Buffet,Bus Station,Bus Stop,Café,Coffee Shop,Convenience Store,Fast Food Restaurant,Food,General Entertainment,Golf Course,Gym,Gym / Fitness Center,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Laundromat,Lighthouse,Lounge,Playground,Pool Hall,Rental Car Location,Restaurant,Spa,Steakhouse,Theater,Theme Park Ride / Attraction,Theme Restaurant,Tourist Information Center,Tram Station
0,Disney's Beach Club Resort,0.166667,0.00000,0.055556,0.000000,0.00000,0.000000,0.055556,0.000000,0.000000,0.00000,0.000000,0.055556,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.166667,0.000000,0.055556,0.055556,0.055556,0.055556,0.000000,0.000000,0.00000,0.000000,0.0000,0.055556,0.00000,0.055556,0.000000,0.000000,0.000000
1,Disney's Beach Club Villas,0.166667,0.00000,0.055556,0.000000,0.00000,0.000000,0.055556,0.000000,0.000000,0.00000,0.000000,0.055556,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.166667,0.000000,0.055556,0.055556,0.055556,0.055556,0.000000,0.000000,0.00000,0.000000,0.0000,0.055556,0.00000,0.055556,0.000000,0.000000,0.000000
2,Disney's BoardWalk Inn,0.166667,0.00000,0.055556,0.000000,0.00000,0.000000,0.055556,0.000000,0.000000,0.00000,0.000000,0.055556,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.166667,0.000000,0.055556,0.055556,0.055556,0.055556,0.000000,0.000000,0.00000,0.000000,0.0000,0.055556,0.00000,0.055556,0.000000,0.000000,0.000000
3,Disney's BoardWalk Villas,0.166667,0.00000,0.055556,0.000000,0.00000,0.000000,0.055556,0.000000,0.000000,0.00000,0.000000,0.055556,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.166667,0.000000,0.055556,0.055556,0.055556,0.055556,0.000000,0.000000,0.00000,0.000000,0.0000,0.055556,0.00000,0.055556,0.000000,0.000000,0.000000
4,Disney's Caribbean Beach Resort,0.000000,0.00000,0.000000,0.029412,0.00000,0.029412,0.029412,0.058824,0.117647,0.00000,0.029412,0.029412,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.147059,0.029412,0.147059,0.000000,0.029412,0.000000,0.000000,0.029412,0.029412,0.00000,0.058824,0.0000,0.000000,0.00000,0.000000,0.029412,0.029412,0.029412
5,Disney's Riviera Resort,0.062500,0.03125,0.000000,0.000000,0.03125,0.000000,0.000000,0.093750,0.000000,0.03125,0.000000,0.000000,0.03125,0.03125,0.03125,0.03125,0.03125,0.03125,0.062500,0.125000,0.093750,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.03125,0.000000,0.0625,0.000000,0.03125,0.000000,0.000000,0.000000,0.000000
6,Disney's Yacht Club Resort,0.166667,0.00000,0.055556,0.000000,0.00000,0.000000,0.055556,0.000000,0.000000,0.00000,0.000000,0.055556,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.166667,0.000000,0.055556,0.055556,0.055556,0.055556,0.000000,0.000000,0.00000,0.000000,0.0000,0.055556,0.00000,0.055556,0.000000,0.000000,0.000000


#### Let's confirm the new size

In [32]:
epcot_resort_grouped.shape

(7, 37)

#### Let's print each resort hotel along with the top 5 most common venues

In [33]:
num_top_venues = 5

for hood in epcot_resort_grouped['Resort']:
    print("----"+hood+"----")
    temp = epcot_resort_grouped[epcot_resort_grouped['Resort'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Disney's Beach Club Resort----
                 venue  freq
0  American Restaurant  0.17
1            Hotel Bar  0.17
2                Beach  0.06
3           Steakhouse  0.06
4       Ice Cream Shop  0.06


----Disney's Beach Club Villas----
                 venue  freq
0  American Restaurant  0.17
1            Hotel Bar  0.17
2                Beach  0.06
3           Steakhouse  0.06
4       Ice Cream Shop  0.06


----Disney's BoardWalk Inn----
                 venue  freq
0  American Restaurant  0.17
1            Hotel Bar  0.17
2                Beach  0.06
3           Steakhouse  0.06
4       Ice Cream Shop  0.06


----Disney's BoardWalk Villas----
                 venue  freq
0  American Restaurant  0.17
1            Hotel Bar  0.17
2                Beach  0.06
3           Steakhouse  0.06
4       Ice Cream Shop  0.06


----Disney's Caribbean Beach Resort----
         venue  freq
0        Hotel  0.15
1   Hotel Pool  0.15
2     Bus Stop  0.12
3   Restaurant  0.06
4  Bus Station  

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [54]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Resort']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
epcot_resort_venues_sorted = pd.DataFrame(columns=columns)
epcot_resort_venues_sorted['Resort'] = epcot_resort_grouped['Resort']

for ind in np.arange(epcot_resort_grouped.shape[0]):
    epcot_resort_venues_sorted.iloc[ind, 1:] = return_most_common_venues(epcot_resort_grouped.iloc[ind, :], num_top_venues)

epcot_resort_venues_sorted.head()

,Resort,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Disney's Beach Club Resort,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse
1,Disney's Beach Club Villas,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse
2,Disney's BoardWalk Inn,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse
3,Disney's BoardWalk Villas,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse
4,Disney's Caribbean Beach Resort,Hotel,Hotel Pool,Bus Stop,Bus Station,Restaurant,Tram Station,Coffee Shop,Tourist Information Center,Hotel Bar,Laundromat


## 4. Cluster Resorts

Run *k*-means to cluster the resorts.  Changed clusters from 5 to 3 since there are few resorts.

In [36]:
# set number of clusters
kclusters = 3

epcot_resort_grouped_clustering = epcot_resort_grouped.drop('Resort', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(epcot_resort_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[1:10] 

array([1, 1, 1, 0, 2, 1])

In [37]:
epcot_resort_venues_sorted

,Resort,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Disney's Beach Club Resort,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse
1,Disney's Beach Club Villas,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse
2,Disney's BoardWalk Inn,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse
3,Disney's BoardWalk Villas,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse
4,Disney's Caribbean Beach Resort,Hotel,Hotel Pool,Bus Stop,Bus Station,Restaurant,Tram Station,Coffee Shop,Tourist Information Center,Hotel Bar,Laundromat
5,Disney's Riviera Resort,Hotel Bar,Hotel Pool,Bus Station,American Restaurant,Spa,Hotel,Arcade,Boat or Ferry,Café,Fast Food Restaurant
6,Disney's Yacht Club Resort,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each postal code.

In [38]:
epcot_resort_venues_sorted.shape

(7, 11)

In [39]:
epcot_resorts.shape

(7, 7)

In [40]:
epcot_resort_venues_sorted.head()

,Resort,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Disney's Beach Club Resort,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse
1,Disney's Beach Club Villas,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse
2,Disney's BoardWalk Inn,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse
3,Disney's BoardWalk Villas,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse
4,Disney's Caribbean Beach Resort,Hotel,Hotel Pool,Bus Stop,Bus Station,Restaurant,Tram Station,Coffee Shop,Tourist Information Center,Hotel Bar,Laundromat


In [41]:
# add clustering label
epcot_resort_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

epcot_resort_merged = epcot_resorts
epcot_resort_merged.set_index('Resort')

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
epcot_resort_merged = epcot_resort_merged.join(epcot_resort_venues_sorted.set_index('Resort'), on='Resort', how='right')

epcot_resort_merged.head() # check the last columns!

,Resort,Opening Date,Theme,Rooms,Resort Area,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Disney's Beach Club Resort,"November 19, 1990",Newport Beach cottage,576,Epcot,28.37382,-81.55731,1,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse
1,Disney's BoardWalk Inn,"July 1, 1996",Early-20th-century Atlantic and Ocean City,378,Epcot,28.37382,-81.55731,1,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse
2,Disney's Yacht Club Resort,"November 5, 1990",Martha's Vineyard Resort,621,Epcot,28.37382,-81.55731,1,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse
3,Disney's Caribbean Beach Resort,"October 1, 1988",Caribbean Islands,"1,536",Epcot,28.36202,-81.54264,0,Hotel,Hotel Pool,Bus Stop,Bus Station,Restaurant,Tram Station,Coffee Shop,Tourist Information Center,Hotel Bar,Laundromat
4,Disney's Beach Club Villas,"July 1, 2002",Newport resort,282,Epcot,28.37382,-81.55731,1,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse


Finally, let's visualize the resulting clusters

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(epcot_resort_merged['Latitude'], epcot_resort_merged['Longitude'], 
                                  epcot_resort_merged['Resort'], epcot_resort_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [46]:
epcot_resort_merged.loc[epcot_resort_merged['Cluster Labels'] == 0, 
                        epcot_resort_merged.columns[[0] + list(range(3, epcot_resort_merged.shape[1]))]]

,Resort,Rooms,Resort Area,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Disney's Caribbean Beach Resort,"1,536",Epcot,28.36202,-81.54264,0,Hotel,Hotel Pool,Bus Stop,Bus Station,Restaurant,Tram Station,Coffee Shop,Tourist Information Center,Hotel Bar,Laundromat


#### Cluster 2

In [52]:
c2=epcot_resort_merged.loc[epcot_resort_merged['Cluster Labels'] == 1, 
                        epcot_resort_merged.columns[[0] + list(range(3, epcot_resort_merged.shape[1]))]]
c2['Rooms']=c2['Rooms'].astype(int)
print('Total number of rooms for cluster 2 is ', str(c2['Rooms'].sum()))
c2

Total number of rooms for cluster 2 is  2387


,Resort,Rooms,Resort Area,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Disney's Beach Club Resort,576,Epcot,28.373820,-81.557310,1,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse
1,Disney's BoardWalk Inn,378,Epcot,28.373820,-81.557310,1,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse
2,Disney's Yacht Club Resort,621,Epcot,28.373820,-81.557310,1,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse
4,Disney's Beach Club Villas,282,Epcot,28.373820,-81.557310,1,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse
5,Disney's BoardWalk Villas,530,Epcot,28.373819,-81.557312,1,American Restaurant,Hotel Bar,Ice Cream Shop,Convenience Store,Theme Park Ride / Attraction,Beach,Steakhouse,Buffet,Lounge,Lighthouse


#### Cluster 3

In [48]:
epcot_resort_merged.loc[epcot_resort_merged['Cluster Labels'] == 2, 
                        epcot_resort_merged.columns[[0] + list(range(3, epcot_resort_merged.shape[1]))]]

,Resort,Rooms,Resort Area,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Disney's Riviera Resort,300,Epcot,28.37797,-81.51904,2,Hotel Bar,Hotel Pool,Bus Station,American Restaurant,Spa,Hotel,Arcade,Boat or Ferry,Café,Fast Food Restaurant
